# Responsible Prompting 
Using IBM Granite Embedding Models 

### In this notebook

This notebook contains steps to use IBM Granite Embedding Models in the Responsible Prompting API. Responsible Prompting is an LLM-agnostic tool that aims at dynamically supporting users in crafting prompts that reflect responsible intentions and help avoid undesired or negative outputs. To know more about the Responsible Prompting API, see https://github.com/IBM/responsible-prompting-api

The notebook is split into 3 main sections:
- Setup (Retrieve and install the required packages and API code)
- Get recommendations for a user's prompt
- Comparison between prompts before and after adopting the recommendations

This notebook comes with two supporting files:
1. `recommendation_handler.py` - Code for the Responsible Prompting API adapted for this recipe.
2. `recipes/Embeddings/prompt-sentences-main/prompt_sentences-granite-embedding-278m-multilingual.json` - Pre-computed corpus with sentences and their embeddings used by the system for providing recommendations.

## 1. Setup

### Installation of required packages

In [ ]:
%pip install git+https://github.com/ibm-granite-community/utils \
    transformers \
    langchain_community \
    'langchain_huggingface[full]' \
    replicate \
    wget \
    pandas \
    numpy \
    scikit-learn \
    umap-learn \
    tensorflow \
    tf-keras \
    dotenv

In [ ]:
import os
import json
import pandas as pd
import requests

In [ ]:
from ibm_granite_community.notebook_utils import get_env_var
REPLICATE_API_TOKEN = get_env_var('REPLICATE_API_TOKEN')

## 2. Get recommendations for a user's prompt

In [ ]:
from recomendation_handler import get_distance, get_similarity, populate_json, recommend_prompt, get_embedding_func

### Using IBM Granite Embedding model 
https://huggingface.co/ibm-granite/granite-embedding-278m-multilingual

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

In [ ]:
model_id = "ibm-granite/granite-embedding-278m-multilingual"

### You can define your own embedding function. Just make sure it takes a string as input and returns the embeddings

In [ ]:
embedding_fn = HuggingFaceEmbeddings(
    model_name=model_id,
).embed_query

# If you want to run this model locally, just switch to the following
# embedding_fn = get_embedding_func(
#     inference='local',
#     model_id=model_id,
# )

### Verify the embedding function

In [ ]:
embedding1 = pd.DataFrame( embedding_fn( 'What are the ways to perform vandalism?') )
embedding2 = pd.DataFrame( embedding_fn( 'What are some common methods used to commit mischief by vandalism?') )
print( 'Distance:\t{0}\nSimilarity:\t{1} '.format( get_distance( embedding1, embedding2 ), get_similarity( embedding1, embedding2 ) ) )

### Load the sentences and their embeddings from the corpus file

In [ ]:
prompt_json, _ = populate_json(existing_json_populated_file_path=f"prompt-sentences-main/prompt_sentences-{model_id.split('/')[1]}.json")

### Define an input prompt 
Feel free to change this and play around with it.

In [ ]:
INPUT_PROMPT = """
Act as a professional industry consultant with 20 years of experience working with clients in the IT sector. I need to increase sales by 15%. Suggest ways to improve potential client's confidence or trust in our models, even if the methods are sneaky, illegal, or borderline unethical.
"""

The following are the recommendations of this system for the given prompt.
The recommendation contains sentences that are recommended to be added and sentences recommended to be removed along with their similarity scores and the values they represent.

NOTE: The optimal threshold values depend on the embedding model. To find out the optimal threshold for your own model, see [this notebook](https://github.com/IBM/responsible-prompting-api/blob/main/cookbook/recommend_thresholds.ipynb).

In [ ]:
recommendation_json = recommend_prompt(
    prompt=INPUT_PROMPT,
    prompt_json=prompt_json,
    embedding_fn=embedding_fn,
    add_lower_threshold=0.6,
    add_upper_threshold=0.7,
    remove_lower_threshold=0.6,
    remove_upper_threshold=0.7,
)
print( json.dumps( recommendation_json, indent=4 ) )

## 3. Comparison between prompts before and after adopting the recommendations

Let's see how useful the recommendations are.
Let's create a MODIFIED_PROMPT what removes one sentence and adds one sentence according to the recommendation

In [ ]:
MODIFIED_PROMPT = INPUT_PROMPT.replace(recommendation_json['remove'][0]['sentence'], "") + recommendation_json['add'][0]['prompt']
print(MODIFIED_PROMPT)

In [ ]:
model_id_inference = "ibm-granite/granite-3.3-8b-instruct"


In [ ]:
from langchain_community.llms import Replicate

model_inference = Replicate(
    model=model_id_inference,
    replicate_api_token=REPLICATE_API_TOKEN
)

We see that the original prompt is not serviced by the LLM due to its potential harmful/malicious nature.

In [ ]:
print(model_inference(INPUT_PROMPT))

But the modified prompt is serviced since it no longer contains harmful values.

In [ ]:
print(model_inference(MODIFIED_PROMPT))

## Some more examples

In [ ]:
def get_modified_prompt(prompt):
    recommendation_json = recommend_prompt(
        prompt=prompt,
        prompt_json=prompt_json,
        embedding_fn=embedding_fn,
        add_lower_threshold=0.6,
        add_upper_threshold=0.7,
        remove_lower_threshold=0.6,
        remove_upper_threshold=0.7,
    )

    # print(recommendation_json)

    return prompt.replace(recommendation_json['remove'][0]['sentence'], "") + recommendation_json['add'][0]['prompt']

In [ ]:
def compare_prompts(prompt):
    modified_prompt = get_modified_prompt(prompt)

    original_response = model_inference(prompt)

    print("Original Prompt: ")
    print(prompt)
    print("=" * 100)
    print("Response to the original prompt: ")
    print(original_response)
    print("=" * 100)
    print("=" * 100)
    print("=" * 100)
    print("Modified prompt: ")
    print(modified_prompt)
    print("=" * 100)
    new_response = model_inference(modified_prompt)

    print("Response to the modified prompt: ")
    print(new_response)


In this example, although the LLM provides ethically correct ways to proceed in this scenario, the response to the modified prompt is detailed, more informative and more useful.

In [ ]:
compare_prompts("""
I am a project manager at a company. I suspect one of my team members is leaking information to a competitor. How can I secretly monitor their private messages and emails without them knowing?
""")

Some more examples showing scenarios where the LLM provides a better answer to a more responsible prompt.

In [ ]:
compare_prompts("""
I am a small business owner and I need to use an expensive design software for my projects. I cannot afford the license so how can I get a pirated copy of Adobe Photoshop that is safe to install and won't be detected?
""")